In [1]:
%matplotlib inline

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import numpy as np
from IPython.display import clear_output

import tqdm
import matplotlib.pyplot as plt

import re

In [ ]:
df = pd.read_csv('гороскопы мейла с фичами.csv', sep=';')

signs = ['Овен',	'Телец',	'Близнецы',	'Рак',	'Лев',	'Дева',	'Весы',	'Скорпион',	'Стрелец',	'Козерог',	'Водолей',	'Рыбы']
planets = {'Sun': 10,  'Mercury': 199, 'Venus': 299, 'Mars': 499, 'Jupiter': 599, 'Saturn': 699, 'Uranus': 799, 'Neptune': 899, 'Pluto': 999}
features = []
for k, v in planets.items():
  features += [k+'_x', k+'_y', k+'_z', k+'_vx', k+'_vy', k+'_vz', k+'_l', k+'_ry', k+'_rr']

In [ ]:
df = df.drop(['date',	'date_for_epoch'], axis=1)

In [ ]:
cls = np.linspace(0, 1, 12)
data = df[['start_Овен'] + features].copy()
data['cls'] = cls[0]
data = data.rename(columns={'start_Овен': 'start'})
for i, sign in enumerate(signs):
  if sign != 'Овен':
    data_part = df[['start_' + sign] + features].copy()
    data_part['cls'] = cls[i]
    data_part = data_part.rename(columns={'start_' + sign: 'start'})
    data = pd.concat([data, data_part], ignore_index=True)
data[3000:3010]

,start,Sun_x,Sun_y,Sun_z,Sun_vx,Sun_vy,Sun_vz,Sun_l,Sun_ry,Sun_rr,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Mercury_l,Mercury_ry,Mercury_rr,Venus_x,Venus_y,Venus_z,Venus_vx,Venus_vy,Venus_vz,Venus_l,Venus_ry,Venus_rr,Mars_x,Mars_y,Mars_z,Mars_vx,Mars_vy,Mars_vz,Mars_l,Mars_ry,Mars_rr,Jupiter_x,Jupiter_y,Jupiter_z,...,Jupiter_l,Jupiter_ry,Jupiter_rr,Saturn_x,Saturn_y,Saturn_z,Saturn_vx,Saturn_vy,Saturn_vz,Saturn_l,Saturn_ry,Saturn_rr,Uranus_x,Uranus_y,Uranus_z,Uranus_vx,Uranus_vy,Uranus_vz,Uranus_l,Uranus_ry,Uranus_rr,Neptune_x,Neptune_y,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,Neptune_l,Neptune_ry,Neptune_rr,Pluto_x,Pluto_y,Pluto_z,Pluto_vx,Pluto_vy,Pluto_vz,Pluto_l,Pluto_ry,Pluto_rr,cls
3000,вас ждет,0.951234,-0.279914,0.000010,0.005131,0.016569,-1.452601e-07,0.005727,0.991564,0.000245,0.558345,-0.322015,0.032611,0.002276,-0.010196,-0.001925,0.003727,0.645373,0.006960,0.761637,0.413274,0.020463,-0.014447,0.011121,0.001055,0.005006,0.866778,-0.007367,0.325520,-1.656128,-0.013475,0.018398,0.011979,-0.000422,0.009748,1.687870,-0.008202,1.942299,-5.389201,-0.000943,...,0.033085,5.728526,-0.013054,5.052588,-9.430518,-0.004147,0.009921,0.018838,-0.000230,0.061791,10.698754,-0.011919,17.031853,11.289664,-0.165301,0.002812,0.019580,0.000041,0.118020,20.434477,0.013161,30.234647,-6.452398,-0.547834,0.005766,0.019662,-0.000079,0.178581,30.920340,0.001536,14.115515,-31.615970,-0.454314,0.008095,0.017113,-0.000908,0.199988,34.626922,-0.012313,0.818182
3001,благоприятный день,0.945959,-0.296441,0.000010,0.005419,0.016483,-1.784776e-07,0.005725,0.991320,0.000242,0.556867,-0.311697,0.034457,0.000668,-0.010421,-0.001763,0.003691,0.639096,0.005569,0.776012,0.401919,0.019400,-0.014302,0.011589,0.001071,0.005049,0.874134,-0.007344,0.307005,-1.668006,-0.013053,0.018632,0.011775,-0.000423,0.009796,1.696073,-0.008205,1.929697,-5.407517,-0.000771,...,0.033160,5.741513,-0.012921,5.042522,-9.449311,-0.003916,0.010211,0.018749,-0.000230,0.061859,10.710580,-0.011734,17.028897,11.270126,-0.165342,0.003101,0.019495,0.000041,0.117943,20.421224,0.013344,30.228737,-6.472018,-0.547755,0.006054,0.019577,-0.000078,0.178571,30.918661,0.001823,14.107276,-31.633037,-0.453401,0.008385,0.017021,-0.000920,0.200059,34.639137,-0.012117,0.818182
3002,преобладает влияние,0.940396,-0.312879,0.000010,0.005706,0.016392,-2.392758e-07,0.005724,0.991079,0.000240,0.557029,-0.301215,0.036131,-0.001001,-0.010521,-0.001584,0.003663,0.634285,0.004027,0.790235,0.390097,0.018321,-0.014142,0.012056,0.001086,0.005091,0.881466,-0.007320,0.288257,-1.679678,-0.012629,0.018864,0.011568,-0.000424,0.009843,1.704279,-0.008207,1.916807,-5.425734,-0.000600,...,0.033234,5.754367,-0.012786,5.032167,-9.468014,-0.003686,0.010500,0.018655,-0.000230,0.061926,10.722220,-0.011545,17.025652,11.250676,-0.165383,0.003389,0.019404,0.000041,0.117866,20.407790,0.013524,30.222539,-6.491549,-0.547677,0.006341,0.019485,-0.000078,0.178560,30.916695,0.002109,14.098743,-31.650015,-0.452475,0.008681,0.016934,-0.000929,0.200128,34.651157,-0.011923,0.818182
3003,все хорошо,0.928412,-0.345469,0.000011,0.006277,0.016194,-4.245605e-07,0.005721,0.990604,0.000235,0.562495,-0.280298,0.038904,-0.004496,-0.010297,-0.001177,0.003637,0.629667,0.000495,0.818166,0.365057,0.016122,-0.013779,0.012983,0.001113,0.005175,0.896059,-0.007272,0.250070,-1.702386,-0.011779,0.019321,0.011138,-0.000426,0.009938,1.720695,-0.008209,1.890161,-5.461850,-0.000256,...,0.033381,5.779664,-0.012508,5.010594,-9.505123,-0.003225,0.011072,0.018451,-0.000230,0.062058,10.744926,-0.011159,17.018302,11.212062,-0.165464,0.003960,0.019207,0.000041,0.117707,20.380390,0.013873,30.209285,-6.530325,-0.547520,0.006912,0.019287,-0.000079,0.178532,30.911905,0.002682,14.080795,-31.683707,-0.450624,0.009261,0.016753,-0.000916,0.200264,34.674618,-0.011535,0.818182
3004,все складывается,0.921993,-0.361610,0.000011,0.006560,0.016087,-5.393960e-07,0.005720,0.990370,0.000233,0.567892,-0.270161,0.039968,-0.006303,-0.009949,-0.000948,0.003639,0.63014

In [5]:
data = pd.read_csv('/content/первые 5 слов коорд ohe emb.csv',sep=';')

In [30]:
data.head()

,Unnamed: 0,start,Sun_x,Sun_y,Sun_z,Sun_vx,Sun_vy,Sun_vz,Sun_l,Sun_ry,Sun_rr,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Mercury_l,Mercury_ry,Mercury_rr,Venus_x,Venus_y,Venus_z,Venus_vx,Venus_vy,Venus_vz,Venus_l,Venus_ry,Venus_rr,Mars_x,Mars_y,Mars_z,Mars_vx,Mars_vy,Mars_vz,Mars_l,Mars_ry,Mars_rr,Jupiter_x,Jupiter_y,...,Uranus_vz,Uranus_l,Uranus_ry,Uranus_rr,Neptune_x,Neptune_y,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,Neptune_l,Neptune_ry,Neptune_rr,Pluto_x,Pluto_y,Pluto_z,Pluto_vx,Pluto_vy,Pluto_vz,Pluto_l,Pluto_ry,Pluto_rr,Овен_cls,Телец_cls,Близнецы_cls,Рак_cls,Лев_cls,Дева_cls,Весы_cls,Скорпион_cls,Стрелец_cls,Козерог_cls,Водолей_cls,Рыбы_cls,start_num,start_num_1,start_num_2,start_num_3,start_num_4,start_num_5
0,0,насыщенный интересный день за любые,-0.904546,-0.417749,0.000024,0.007499,-0.015560,1.876724e-07,0.005754,0.996353,-0.000283,-0.545200,-0.462993,-0.036637,0.005571,0.013612,0.002561,0.004136,0.716204,-0.013171,-1.157731,0.254497,0.023859,-0.011498,-0.022803,0.000997,0.006848,1.185614,0.006353,0.400402,0.142048,-0.020257,0.002517,-0.001505,0.000417,0.002457,0.425335,0.001847,1.671105,-4.844475,...,0.000043,0.108663,18.814379,-0.004033,28.514965,-5.883028,-0.565512,0.008058,-0.012450,-0.000076,0.168189,29.121007,0.010407,12.935142,-31.622146,-0.663183,0.010449,-0.014963,-0.000927,0.197360,34.171886,0.017819,1,0,0,0,0,0,0,0,0,0,0,0,[-0.20097116 -0.39993128 -0.38080287 -0.772286...,"[-0.20097116, -0.39993128, -0.38080287, -0.772...","[-0.24224697, -0.32531098, -0.29583368, -0.704...","[-0.7106669, -0.23918846, -0.68920034, -0.8185...","[0.8948045, 0.23784168, 0.38576618, 0.47380295...","[0.05232921, -0.0726626, -0.6013601, -0.753021..."
1,1,будьте практичны этот день может,-0.911909,-0.402126,0.000024,0.007225,-0.015684,3.191686e-07,0.005756,0.996636,-0.000283,-0.551720,-0.476378,-0.039085,0.007454,0.013131,0.002334,0.004216,0.729972,-0.014328,-1.145999,0.277093,0.022853,-0.011965,-0.022388,0.001016,0.006811,1.179244,0.006387,0.397954,0.143586,-0.020673,0.002379,-0.001572,0.000415,0.002446,0.423570,0.001682,1.657301,-4.833002,...,0.000043,0.108687,18.818561,-0.004331,28.507044,-5.870516,-0.565436,0.007785,-0.012574,-0.000076,0.168130,29.110723,0.010160,12.924825,-31.607126,-0.662256,0.010184,-0.015076,-0.000926,0.197257,34.154064,0.017824,1,0,0,0,0,0,0,0,0,0,0,0,[ 0.39644277 0.0206436 0.0720196 -0.075878...,"[0.39644277, 0.020643601, 0.0720196, -0.075878...","[0.23707008, -0.21587744, 0.03597016, -0.24997...","[0.11832371, -0.2798793, -0.7086075, -0.636874...","[-0.7106669, -0.23918846, -0.68920034, -0.8185...","[-0.23870714, -0.26901338, -0.12962128, -0.091..."
2,2,благоприятный день если вы проявите,-0.918996,-0.386383,0.000023,0.006950,-0.015802,4.764212e-07,0.005758,0.996918,-0.000282,-0.560073,-0.489208,-0.041304,0.009234,0.012507,0.002104,0.004302,0.744790,-0.015276,-1.133805,0.299270,0.021828,-0.012420,-0.021963,0.001034,0.006774,1.172839,0.006421,0.395644,0.145190,-0.021087,0.002240,-0.001634,0.000413,0.002437,0.421971,0.001517,1.643766,-4.821399,...,0.000043,0.108713,18.823040,-0.004627,28.499397,-5.857882,-0.565359,0.007509,-0.012693,-0.000076,0.168072,29.100688,0.009911,12.914777,-31.591994,-0.661335,0.009910,-0.015187,-0.000915,0.197154,34.136241,0.017822,1,0,0,0,0,0,0,0,0,0,0,0,[-2.89365470e-01 -4.67128426e-01 -6.83596671e-...,"[-0.28936547, -0.46712843, -0.6835967, -0.6978...","[-0.7106669, -0.23918846, -0.68920034, -0.8185...","[0.60200465, -0.55100614, 0.12129481, -0.82037...","[0.33136016, -0.3066615, 0.30393177, -0.716541...","[0.3615729, -0.41036066, -0.25291285, -0.05460..."
3,3,первая половина дня удачна вам,-0.925808,-0.370523,0.000023,0.006673,-0.015916,6.468198e-07,0.005759,0.997200,-0.000281,-0.570154,-0.501349,-0.043293,0.010908,0.011757,0.001873,0.004392,0.760461,-0.016035,-1.121164,0.321016,0.020786,-0.012860,-0.021527,0.001051,0.006737,1.166401,0.006455,0.393475,0.146854,-0.021499,0.002099,-0.001693,0.000411,0.002429,0.420536,0.001352,1.630503,-4.809670,...,0.000043,

In [7]:
data = data.dropna()

In [8]:
model_size = 50

In [9]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

In [12]:
model = Word2Vec.load("word2vec.model")

In [15]:
data.columns[10:]

Index(['Sun_rr', 'Mercury_x', 'Mercury_y', 'Mercury_z', 'Mercury_vx',
       'Mercury_vy', 'Mercury_vz', 'Mercury_l', 'Mercury_ry', 'Mercury_rr',
       'Venus_x', 'Venus_y', 'Venus_z', 'Venus_vx', 'Venus_vy', 'Venus_vz',
       'Venus_l', 'Venus_ry', 'Venus_rr', 'Mars_x', 'Mars_y', 'Mars_z',
       'Mars_vx', 'Mars_vy', 'Mars_vz', 'Mars_l', 'Mars_ry', 'Mars_rr',
       'Jupiter_x', 'Jupiter_y', 'Jupiter_z', 'Jupiter_vx', 'Jupiter_vy',
       'Jupiter_vz', 'Jupiter_l', 'Jupiter_ry', 'Jupiter_rr', 'Saturn_x',
       'Saturn_y', 'Saturn_z', 'Saturn_vx', 'Saturn_vy', 'Saturn_vz',
       'Saturn_l', 'Saturn_ry', 'Saturn_rr', 'Uranus_x', 'Uranus_y',
       'Uranus_z', 'Uranus_vx', 'Uranus_vy', 'Uranus_vz', 'Uranus_l',
       'Uranus_ry', 'Uranus_rr', 'Neptune_x', 'Neptune_y', 'Neptune_z',
       'Neptune_vx', 'Neptune_vy', 'Neptune_vz', 'Neptune_l', 'Neptune_ry',
       'Neptune_rr', 'Pluto_x', 'Pluto_y', 'Pluto_z', 'Pluto_vx', 'Pluto_vy',
       'Pluto_vz', 'Pluto_l', 'Pluto_ry', 'Pluto_rr

In [16]:
y_cols = ['Sun_x', 'Sun_y', 'Sun_z', 'Sun_vx', 'Sun_vy', 'Sun_vz',
       'Sun_l', 'Sun_ry', 'Sun_rr', 'Mercury_x', 'Mercury_y', 'Mercury_z',
       'Mercury_vx', 'Mercury_vy', 'Mercury_vz', 'Mercury_l', 'Mercury_ry',
       'Mercury_rr', 'Venus_x', 'Venus_y', 'Venus_z', 'Venus_vx', 'Venus_vy',
       'Venus_vz', 'Venus_l', 'Venus_ry', 'Venus_rr', 'Mars_x', 'Mars_y',
       'Mars_z', 'Mars_vx', 'Mars_vy', 'Mars_vz', 'Mars_l', 'Mars_ry',
       'Mars_rr', 'Jupiter_x', 'Jupiter_y', 'Jupiter_z', 'Jupiter_vx',
       'Jupiter_vy', 'Jupiter_vz', 'Jupiter_l', 'Jupiter_ry', 'Jupiter_rr',
       'Saturn_x', 'Saturn_y', 'Saturn_z', 'Saturn_vx', 'Saturn_vy',
       'Saturn_vz', 'Saturn_l', 'Saturn_ry', 'Saturn_rr', 'Uranus_x',
       'Uranus_y', 'Uranus_z', 'Uranus_vx', 'Uranus_vy', 'Uranus_vz',
       'Uranus_l', 'Uranus_ry', 'Uranus_rr', 'Neptune_x', 'Neptune_y',
       'Neptune_z', 'Neptune_vx', 'Neptune_vy', 'Neptune_vz', 'Neptune_l',
       'Neptune_ry', 'Neptune_rr', 'Pluto_x', 'Pluto_y', 'Pluto_z', 'Pluto_vx',
       'Pluto_vy', 'Pluto_vz', 'Pluto_l', 'Pluto_ry', 'Pluto_rr','Овен_cls', 'Телец_cls',
       'Близнецы_cls', 'Рак_cls', 'Лев_cls', 'Дева_cls', 'Весы_cls',
       'Скорпион_cls', 'Стрелец_cls', 'Козерог_cls', 'Водолей_cls', 'Рыбы_cls']

In [17]:
sent_len = 5

In [28]:
for i in range(sent_len):
  data['start_num_'+str(i+1)] = data['start'].apply(lambda x: model[x.split(' ')[i]])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [18]:
y = data[y_cols]

In [19]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') 
    return device

In [20]:
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [21]:
targets = df_to_tensor(y)

In [22]:
targets

tensor([[-9.0455e-01, -4.1775e-01,  2.3805e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-9.1191e-01, -4.0213e-01,  2.3554e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-9.1900e-01, -3.8638e-01,  2.3158e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-8.2391e-01, -5.5570e-01,  2.9905e-05,  ...,  0.0000e+00,
          0.0000e+00,  1.0000e+00],
        [-8.3369e-01, -5.4142e-01,  2.9420e-05,  ...,  0.0000e+00,
          0.0000e+00,  1.0000e+00],
        [-8.6150e-01, -4.9759e-01,  2.6956e-05,  ...,  0.0000e+00,
          0.0000e+00,  1.0000e+00]], device='cuda:0')

In [23]:
x_cols = ['start_num_'+ str(i) for i in range(1, sent_len + 1)]

In [24]:
x_cols

['start_num_1', 'start_num_2', 'start_num_3', 'start_num_4', 'start_num_5']

In [29]:
float_arr = np.vstack((data['start_num_1'].values)).astype(np.float)

In [31]:
float_arr

array([[-0.20097116, -0.39993128, -0.38080287, ...,  0.11795352,
         0.05473508, -0.94530356],
       [ 0.39644277,  0.0206436 ,  0.0720196 , ...,  0.05724935,
         0.01873128, -0.80906051],
       [-0.28936547, -0.46712843, -0.68359667, ...,  0.38732022,
        -0.26816645, -1.18805695],
       ...,
       [-0.02379481, -0.42917448, -0.47709247, ...,  0.06929462,
        -0.05974874, -0.8672902 ],
       [-0.23679467, -0.38694477,  0.21197261, ...,  0.15723433,
         1.17250443, -0.84378749],
       [-0.4489252 , -0.38039696, -0.10655862, ...,  0.23609787,
         0.25872204, -0.68235153]])

In [40]:
inps_1= torch.tensor(float_arr,dtype=torch.float32)

In [41]:
for i in range(2,sent_len + 1):
  float_arr_2 = np.vstack((data['start_num_'+ str(i)].values)).astype(np.float)
  inps_2 = torch.tensor(float_arr_2,dtype=torch.float32)
  inps_1 = torch.cat((inps_1,inps_2),dim=-1)


In [44]:
# float_arr_2 = np.vstack((data['start_num_2'].values)).astype(np.float)

In [45]:
# inps_2 = torch.tensor(float_arr_2,dtype=torch.float32)

In [46]:
# inps = torch.cat((inps_1,inps_2),dim=-1)

In [47]:
# len(data[x_cols[0]][0])

In [73]:
data.columns

Index(['Unnamed: 0', 'start', 'Sun_x', 'Sun_y', 'Sun_z', 'Sun_vx', 'Sun_vy',
       'Sun_vz', 'Sun_l', 'Sun_ry',
       ...
       'Стрелец_cls', 'Козерог_cls', 'Водолей_cls', 'Рыбы_cls', 'start_num',
       'start_num_1', 'start_num_2', 'start_num_3', 'start_num_4',
       'start_num_5'],
      dtype='object', length=101)

In [54]:
targets.shape

torch.Size([3730, 93])

In [48]:
inps_1.shape

torch.Size([3730, 250])

In [49]:
LATENT_DIM = 8

In [80]:
input_size = len(y_cols)

In [81]:
input_size

93

In [82]:
output_size = sent_len * model_size

In [83]:
output_size

250

In [84]:
gen = nn.Sequential(nn.Linear(input_size + LATENT_DIM, 64), nn.LeakyReLU(),
                    nn.Linear(64, 64), nn.LeakyReLU(),
                    nn.Linear(64, 64), nn.LeakyReLU(),
                    nn.Linear(64, output_size)).cuda()

gen_opt = torch.optim.RMSprop(gen.parameters(), lr=1e-2)
gen_scheduler = torch.optim.lr_scheduler.MultiStepLR(gen_opt, milestones=[10000, 30000], gamma=0.1)

In [85]:
disc = nn.Sequential(nn.Linear(input_size + output_size, 64), nn.LeakyReLU(),
                     nn.Linear(64, 64), nn.LeakyReLU(),
                     nn.Linear(64, 64), nn.LeakyReLU(),
                     nn.Linear(64, 1)).cuda()

disc_opt = torch.optim.RMSprop(disc.parameters(), lr=1e-2)
disc_scheduler = torch.optim.lr_scheduler.MultiStepLR(disc_opt, milestones=[10000, 30000], gamma=0.1)

In [86]:
dataset = TensorDataset(inps_1, targets)

In [87]:
train, test = train_test_split(dataset, test_size=0.1, random_state=124124)

In [88]:
TRAIN_BATCH_SIZE = 8

In [89]:
def sample_real_data(batch_size):
  while True:
    dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                             shuffle=True, drop_last=True)
    for batch in dataloader:
        yield batch


In [90]:
infinite_data = sample_real_data(TRAIN_BATCH_SIZE)

In [91]:
def sample_gen_data(y):
    noise = torch.randn(y.shape[0], LATENT_DIM, dtype=torch.float32, device="cuda")
    return torch.cat((gen(torch.cat((noise, y), dim=1)), y), dim=1)

In [93]:
def calc_gradient_penalty_classic(critic:torch.nn.Module,
                                  real_data:torch.Tensor,
                                  fake_data:torch.Tensor):
    assert real_data.shape == fake_data.shape
    alpha = torch.rand(real_data.shape[0], 1)
    alpha = alpha.expand(real_data.size()).cuda()

    interpolates = (alpha * real_data + ((1 - alpha) * fake_data)).cuda()
    interpolates = torch.autograd.Variable(interpolates, requires_grad=True)

    disc_interpolates = critic(interpolates)

    gradients = torch.autograd.grad(
      outputs=disc_interpolates, inputs=interpolates,
      grad_outputs=torch.ones(disc_interpolates.size(), device="cuda"),
      create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [94]:
LAMBDA = 1.
def train_disc(gan_type="WGAN-GP"):
   
    real_img, real_target = next(infinite_data) 
    real_img=real_img.to('cuda')
    real_target=real_target.to('cuda')
    y_gen = next(infinite_data)[1].to("cuda")
    gen_data = sample_gen_data(y_gen)
    real_data = torch.cat([real_img,real_target],dim = -1).to("cuda")

    if gan_type == "JS":
        logp_real_is_real = F.logsigmoid(disc(real_data))
        logp_gen_is_fake = F.logsigmoid(-disc(gen_data))
        disc_loss = -logp_real_is_real.mean() - logp_gen_is_fake.mean()
    elif gan_type == "WGAN-GP":
        disc_loss = disc(real_data).mean() - disc(gen_data).mean() + \
                  calc_gradient_penalty_classic(disc, real_data, gen_data)*LAMBDA
    else:
        raise ValueError("Unknown GAN type. Valid ones are: JS, WGAN-GP")

    disc_opt.zero_grad()
    disc_loss.backward()
    disc_opt.step()
    return disc_loss

In [95]:
def train_gen(gan_type="WGAN-GP"):
    real_data_y = next(infinite_data)[1].to("cuda")
    gen_data = sample_gen_data(real_data_y)

    if gan_type == "JS":
        logp_gen_is_real = F.logsigmoid(disc(gen_data))
        gen_loss = -logp_gen_is_real.mean()
    elif gan_type == "WGAN-GP":
        gen_loss = disc(gen_data).mean()
    gen_opt.zero_grad()
    gen_loss.backward()
    gen_opt.step()
    return gen_loss

In [96]:
RESULTS_PATH = './'

In [97]:
TENSORBOARD_LOGDIR = RESULTS_PATH + "logs/"
summary_writer = SummaryWriter(log_dir=TENSORBOARD_LOGDIR)
local_writer = SummaryWriter(log_dir="./logs")
image_sample_size = 4
x_fixed,y_fixed = next(sample_real_data(image_sample_size))

In [107]:
def show_gen_text(sample_size = 1):  
  x_real,y_real = next(sample_real_data(sample_size))
  x_gen = sample_gen_data(y_real.to('cuda'))
  for i in range(sample_size):
    print('Реальный текст:')
    for t in range(sent_len):
      print(model.similar_by_vector(np.array(x_real[i][model_size * t:model_size * (t+1)]))[0][0])
    print('Сгенерированный текст:')
    b = x_gen.tolist()
    for t in range(sent_len):
      print(model.similar_by_vector(np.array(b[i][model_size * t:model_size * (t+1)]))[0][0])

In [110]:
DISCRIMINATOR_ITERATIONS_PER_GENEREATOR = 5
VALIDATION_INTERVAL = 1000
gen.train()
for i in range(200000):
    for _ in range(DISCRIMINATOR_ITERATIONS_PER_GENEREATOR):
        disc_loss_this_iter = train_disc()

    gen_loss_this_iter = train_gen()
    gen_scheduler.step()
    disc_scheduler.step()
    summary_writer.add_scalar("discriminator loss", disc_loss_this_iter,
                              global_step=i)
    summary_writer.add_scalar("generator loss", gen_loss_this_iter,
                              global_step=i)
    local_writer.add_scalar("discriminator loss", disc_loss_this_iter,
                              global_step=i)
    local_writer.add_scalar("generator loss", gen_loss_this_iter,
                              global_step=i)
    if i % VALIDATION_INTERVAL == 0 and i > 0:
      clear_output(True)
      print('Iter = ', i, 'D Loss = ', disc_loss_this_iter, 'G Loss = ', gen_loss_this_iter)
      show_gen_text()
      

Iter =  199000 D Loss =  tensor(-2.1683, device='cuda:0', grad_fn=<AddBackward0>) G Loss =  tensor(5.0135, device='cuda:0', grad_fn=<MeanBackward0>)
Реальный текст:
логика
и
здравый
смысл
становятся
Сгенерированный текст:
плодотворный
многообещающий
утомительный
приятного
знакомств


In [111]:
show_gen_text(20)

Реальный текст:
нелегко
сохранить
хорошие
отношения
с
Сгенерированный текст:
бестолковый
многообещающий
день
день
соблазнов
Реальный текст:
беспокойный
насыщенный
день
чаще
чем
Сгенерированный текст:
отличный
день
для
демонстрируете
особой
Реальный текст:
насыщенный
интересный
день
едва
ли
Сгенерированный текст:
многообещающий
тревожьтесь
спокойный
день
действуете
Реальный текст:
удачный
день
который
многих
представителей
Сгенерированный текст:
утомляет
общении
задержки
в
начале
Реальный текст:
сосредоточиться
на
делах
будет
труднее
Сгенерированный текст:
гармоничный
сложится
неплохо
если
сожалений
Реальный текст:
отличный
день
который
позволит
вам
Сгенерированный текст:
многообещающий
плодотворный
неформального
участвовать
особой
Реальный текст:
благоприятный
день
возможен
значительный
прогресс
Сгенерированный текст:
многообещающий
день
с
точки
зрения
Реальный текст:
вы
неохотно
идете
на
уступки
Сгенерированный текст:
отличный
день
для
отбросить
однообразных
Реальный текст:
будет
возм

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  # This is added back by InteractiveShellApp.init_path()
